## Семинар 1 Индекс

## Intro

### чтение файла 
- конструкция __with open__ (recommended)
- конструкция __open + close__

In [211]:
fpath = 'fpath.txt'

# одним массивом  
with open(fpath, 'r') as f:  
    text = f.read() 

#по строкам, в конце каждой строки \n  
with open(fpath, 'r') as f:   
    text = f.readlines() 

#по строкам, без \n   
with open(fpath, 'r') as f:   
    text = f.read().splitlines() 
    
#not reccomended  
file = open(txt_fpath, 'r')  
text = file.read()    
file.close() 

### работа с файлами и папками

### os.path  
путь до файла

In [ ]:
import os

# возвращает полный путь до папки/файла по имени файла / папки
print(os.path.abspath('fpath.txt'))

# возвращает имя файла / папки по полному пути до него
print(os.path.basename('/your/path/to/folder/with/fpath.txt'))

# проверить существование директории - True / False
print(os.path.exists('your/path/to/any/folder/'))

### os.listdir  
возвращает список файлов в данной директории

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'
os.listdir(main_dir)

сделаем пути абсолютными, чтобы наш код не зависел от того, где лежит этот файл

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir)]

не забывайте исключать системные директории, такие как .DS_Store

In [ ]:
[main_dir + fpath for fpath in os.listdir(main_dir) if not '.DS_Store' in fpath]

### os.walk
root - начальная директория  
dirs - список поддиректорий (папок)   
files - список файлов в этих поддиректориях  

In [ ]:
main_dir = '/your/path/to/folder/with/folders/'

for root, dirs, files in os.walk(main_dir):
    for name in files:
        print(os.path.join(root, name))

> __os.walk__ возвращает генератор, это значит, что получить его элементы можно только проитерировавшись по нему  
но его легко можно превратить в list и увидеть все его значения

In [ ]:
list(os.walk(main_dir))

##  Обратный индекс 

Сам по себе обратный индекс не может осуществлять поиск, для этого необходимо добавить к нему определенную метрику. Это не совсем очевидная задача, поэтому немного отложим ее. А сейчас посмотрим, что полезного можно вытащить из индекса.    
По сути, индекс - это информация о частоте встречаемости слова в каждом документе.   
Из этого можно понять, например:
1. какое слово является самым часто употребимым / редким
2. какие слова встречаются всегда вместе. Так можно парсить твиттер, fb, форумы и отлавливать новые устойчивые выражения в речи
3. какой документ является самым большим / маленьким (очень изощренный способ, когда есть _len_)

### __Задача__: 
получите обратный индекс для коллекция документов.    
Перед этим постройте матрицу терм-документ и сделайте функцию булева поиска, которая по запросу будет возвращать 5 релевантных документов.   
В качестве коллекции возьмите сценарий сезонов сериала Друзья. Одна серия - один документ.

[download_friends_corpus](https://yadi.sk/d/k_M7n63A3adGSz)

Этапы:   
    1. получить коллекцию документов
    2. для каждого файла коллекции сделать необходимую на ваш взгляд предобработку
    3. получить матрицу терм-документ, написать функцию поиска по ней
    4. получить обратный индекс в виде словаря, где ключ - нормализованное слово, 
    значение - список файлов, в которых это слово встречается
    5. вывести кусочек индекса в виде таблицы 
    6. сделать анализ обратного индекса. Это задание принимается в виде кода и ответов на вопросы

![](Friends/wedding.png)

Напоминание:    
> При итерации по списку вы можете помимо самого элемента получить его порядковый номер    
``` for i, element in enumerate(your_list): ...  ```    
Иногда для получения элемента делают так -  ``` your_list[i] ```, старайтесь этого избегать

In [1]:
import os
import re
import pandas as pd
from collections import defaultdict

In [2]:
def get_all_txt(main_dir):
    files_list = []
    for root, dirs, files in os.walk(main_dir):
        for name in files:
            if name.endswith('.txt'):
                files_list.append(os.path.join(root, name))
    return files_list

In [3]:
main_dir = 'Friends/'
files_list = get_all_txt(main_dir)

len(files_list)
### пройдитесь по всем папкам коллекции и соберите все пути .txt файлов
### _check : в коллекции должно быть 165 файлов


165

In [4]:
def text_preprocessing(text):
    text = re.sub('[^А-Яа-яЁё]', ' ', text)
    text = re.sub('\s+', ' ', text)
    text = text.lower()
    tokens = text.split()
    return tokens

def file_preprocessing(file):
    with open(file, 'r', encoding='utf-8') as f:
        tokens = text_preprocessing(f.read())
    return tokens

def input_text_preprocessing(text):
    return text.lower()

# def input_preprocessing(text):
    

In [5]:
### постройте матрицу терм-документ

def get_term_doc_matrix(text, name, matrix=defaultdict(lambda: defaultdict(int))):
    for word in text:
        matrix[word][name] += 1
    return matrix

In [6]:
def get_td_matrix_from_file_list(file_list):
    td_matrix = defaultdict(lambda: defaultdict(int))
    for file in file_list:
        tokens = file_preprocessing(file)
        td_matrix = get_term_doc_matrix(tokens, file, td_matrix)
    return td_matrix

In [7]:
td_matrix = get_td_matrix_from_file_list(files_list)

In [7]:
def parse_query(query):
    parsed = []
    i = 0
    while i < len(query):
        if query[i] == ')':
            return [parsed] + parse_query(query[i + 1:])
        elif query[i] == '(':
            return parsed + parse_query(query[i + 1:])
        elif query[i] == '&':
            parsed += '&'
            i += 2
        elif i + 2 != len(query) and query[i:i + 3] == 'НЕ ':
            parsed += '!'
            i += 3
        elif i + 3 != len(query) and query[i:i + 4] == 'ИЛИ ':
            parsed += '|'
            i += 4
        elif query[i] == ' ':
            i += 1
        else:
            beg = i
            while i < len(query) and query[i] not in {' ', ')'}:
                i += 1
            parsed.append(query[beg:i].lower())
    return parsed

In [8]:
### напишите функцию булева поиска по построенной матрице
def boolean_search(query, matrix, doc_set) -> set:
    """
    Produces a Boolean search according with the term-document matrix
    :return: list of first 5 relevant documents
    """
    if type(query) == str and query in matrix:
        return set(matrix[query])
    elif type(query) == str:
        return set()
    res = set()
    i = 0
    while i < len(query):
        if query[i] == '&':
            res = res and boolean_search(query[i + 1], matrix, doc_set)
            i += 1
        elif query[i] == '|':
            res = res or boolean_search(query[i + 1], matrix, doc_set)
            i += 1
        elif query[i] == '!':
            res = doc_set - boolean_search(query[i + 1], matrix, doc_set)
            i += 1
        else:
            res = boolean_search(query[i], matrix, doc_set)
        i += 1
    return res


def boolean_search_line(line_query, matrix, doc_set, n=5) -> list:
    return list(boolean_search(parse_query(line_query), matrix, doc_set))[:n]


In [9]:
td_matrix = get_td_matrix_from_file_list(files_list)

In [10]:
#запросы 
input_text = [
    'Моника & Фиби & Рэйчел & Чендлер & Джои & Росс',
    '(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс', 
    '(НЕ Моника) & Фиби & Рэйчел & Чендлер & Джои & (НЕ Росс)',
    'НЕ (Моника ИЛИ Росс)'
]

In [11]:
for q in input_text:
    print(q)
    print(boolean_search_line(q, td_matrix, set(files_list)))

Моника & Фиби & Рэйчел & Чендлер & Джои & Росс
['Friends/Friends - season 1/Friends - 1x16 - The One With Two Parts (1).ru.txt', "Friends/Friends - season 2/Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip.ru.txt", "Friends/Friends - season 6/Friends - 6x13 - The One With Rachel's Sister (1).ru.txt", 'Friends/Friends - season 3/Friends - 3x10 - The One Where Rachel Quits.ru.txt', 'Friends/Friends - season 3/Friends - 3x15 - The One Where Ross And Rachel Take A Break (1).ru.txt']
(Моника ИЛИ Фиби) & Рэйчел & (Чендлер ИЛИ Джои) & Росс
['Friends/Friends - season 1/Friends - 1x16 - The One With Two Parts (1).ru.txt', "Friends/Friends - season 2/Friends - 2x01 - The One With Ross's New Girlfriend.DVDRip.ru.txt", "Friends/Friends - season 6/Friends - 6x13 - The One With Rachel's Sister (1).ru.txt", 'Friends/Friends - season 3/Friends - 3x10 - The One Where Rachel Quits.ru.txt', 'Friends/Friends - season 3/Friends - 3x15 - The One Where Ross And Rachel Take A Break (1).ru.txt']
(НЕ 

<img src="pics/inv_index3.svg" alt="Drawing" style="width: 600px;"/>

Совет для построения обратного индекса: 
> В качестве словаря используйте ``` defaultdict ``` из модуля collections   
Так можно избежать конструкции ``` dict.setdefault(key, default=None) ```

In [12]:
def update_inverted_index(inv_index, tokens, doc_name):
    for token in tokens:
        inv_index[token].add(doc_name)
    return inv_index


def inverted_index(files_list) -> dict:
    """
    Create inverted index by input doc collection
    :return: inverted index
    """
    inv_index = defaultdict(set)
    for file in files_list:
        tokens = file_preprocessing(file)
        inv_index = update_inverted_index(inv_index, set(tokens), file)
    return inv_index

In [13]:
inv_index = inverted_index(files_list)

In [136]:
max_pop = (-1, '')
min_pop = (-1, [])
for word in td_matrix:
    word_pop = 0
    for d in td_matrix[word]:
        word_pop += td_matrix[word][d]
    if max_pop[0] == -1:
        max_pop = (word_pop, word)
        min_pop = (word_pop, [word])
    elif max_pop[0] < word_pop:
        max_pop = (word_pop, word)
    elif min_pop[0] > word_pop:
        min_pop = (word_pop, [word])
    elif min_pop[0] == word_pop:
        min_pop[1].append(word)

какое слово является самым частотным?

In [137]:
sorted(td_matrix, key=lambda x: sum([td_matrix[x][i] for i in td_matrix[x]]), reverse=True)[0]

'я'

In [135]:
max_pop

(13014, ['я'])

какое самым редким?

In [138]:
min_pop

(1,
 ['дремлешь',
  'позах',
  'выглядевший',
  'лиотта',
  'причешусь',
  'мельчайшие',
  'мобиле',
  'громкий',
  'дозвонился',
  'родственникам',
  'надеясь',
  'вольёшься',
  'факсовую',
  'дырочек',
  'постирал',
  'телефонами',
  'вторым',
  'приютах',
  'самоубийсва',
  'нехорошем',
  'рассуждаешь',
  'корысти',
  'бескорыстные',
  'корыстной',
  'рушу',
  'победю',
  'преследовать',
  'родных',
  'клаусе',
  'дружка',
  'упоминала',
  'поминала',
  'официанта',
  'взаимоотношениями',
  'уговорю',
  'сюзи',
  'ожидаю',
  'колинз',
  'пробралась',
  'сгребла',
  'опавшие',
  'листья',
  'веранды',
  'застукал',
  'закусывать',
  'печёшь',
  'печке',
  'скрытность',
  'рассудок',
  'вылетаю',
  'агенстве',
  'сковородка',
  'утк',
  'блевал',
  'гусеницами',
  'уильямс',
  'сонома',
  'шимплер',
  'приютский',
  'храброй',
  'укуса',
  'кадре',
  'солей',
  'добровольцам',
  'милость',
  'божию',
  'стесняйтесь',
  'принимаются',
  'рассчитывай',
  'марафон',
  'убивали',
  'жертв

какой набор слов есть во всех документах коллекции?

In [116]:
in_all_docs = []

for word in inv_index:
    if len(inv_index[word]) == 165:
        in_all_docs.append(word)

in_all_docs

['не',
 'ты',
 'он',
 'если',
 'это',
 'а',
 'и',
 'я',
 'тебе',
 'у',
 'она',
 'мне',
 'на',
 'но',
 'о',
 'как',
 'мы',
 'то',
 'нет',
 'с',
 'просто',
 'так',
 'да',
 'меня',
 'тебя',
 'что',
 'ну',
 'в']

In [121]:
def get_season_number(line):
    res = re.search('(\d)x\d{2}', line)
    return res.group(1)

In [123]:
def get_word_season_popularity(name, matrix):
    season_pop = defaultdict(int)
    for doc in matrix[name]:
        season_pop[get_season_number(doc)] += matrix[name][doc]
    return season_pop

какой сезон был самым популярным у Чендлера? 

6

In [141]:
get_word_season_popularity('чендлер', td_matrix)

defaultdict(int,
            {'5': 106, '3': 58, '7': 91, '4': 74, '1': 48, '6': 109, '2': 48})

у Моники?

5

In [142]:
get_word_season_popularity('моника', td_matrix)

defaultdict(int,
            {'5': 94, '3': 56, '7': 86, '4': 43, '1': 46, '6': 82, '2': 64})

кто из главных героев статистически самый популярный?

Росс

In [144]:
characters = ['Моника', 'Фиби', 'Рэйчел', 'Чендлер', 'Джои', 'Росс']
charact_pop = {c.lower(): 0 for c in characters}

In [145]:
for c in characters:
    c = c.lower()
    for doc in td_matrix[c]:
        charact_pop[c] += td_matrix[c][doc]

In [146]:
charact_pop

{'моника': 471,
 'фиби': 575,
 'рэйчел': 237,
 'чендлер': 534,
 'джои': 69,
 'росс': 746}

С помощью обратного индекса произведите следующую аналитику:  

1) общая аналитика
- какое слово является самым частотным?
- какое самым редким?
- какой набор слов есть во всех документах коллекции?

2) частота встречаемости имен главных героев в каждом сезоне      
- какой сезон был самым популярным у Чендлера? у Моники?   
- кто из главных героев статистически самый популярный? 


## Функция ранжирования Okapi BM25

Для обратного индекса есть общепринятая формула для ранжирования *Okapi best match 25* ([Okapi BM25](https://ru.wikipedia.org/wiki/Okapi_BM25)).    
Пусть дан запрос $Q$, содержащий слова  $q_1, ... , q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:

$$ score(D, Q) = \sum_{i}^{n} \text{IDF}(q_i)*\frac{(k_1+1)*f(q_i,D)}{f(q_i,D)+k_1(1-b+b\frac{|D|}{avgdl})} $$ 
где   
>$f(q_i,D)$ - частота слова $q_i$ в документе $D$ (TF)       
$|D|$ - длина документа (количество слов в нём)   
*avgdl* — средняя длина документа в коллекции    
$k_1$ и $b$ — свободные коэффициенты, обычно их выбирают как $k_1$=2.0 и $b$=0.75   
$$$$
$\text{IDF}(q_i)$ есть обратная документная частота (IDF) слова $q_i$: 
$$\text{IDF}(q_i) = \log\frac{N-n(q_i)+0.5}{n(q_i)+0.5},$$
>> где $N$ - общее количество документов в коллекции   
$n(q_i)$ — количество документов, содержащих $q_i$

In [14]:
### реализуйте эту функцию ранжирования 
from math import log

k1 = 2.0
b = 0.75

def score_BM25(qf, dl, avgdl, k1, b, N, n) -> float:
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    idf = log((N - n + 0.5) / (n + 0.5))
    score = idf * (k1 + 1) * qf / (qf + k1 * (1 - b + b * dl / avgdl))
    return score

### __Задача__:    
напишите функцию, которая сортирует поисковую выдачу для любого входящего запроса согласно метрике *Okapi BM25*.    
Выведите 10 первых результатов и их скор по запросу **рождественские каникулы**. 

In [15]:
def get_avgdl(texts):
    N = len(texts)
    sum_len = 0
    for doc in texts:
        sum_len += len(texts[doc])
    return sum_len / N

In [16]:
def get_texts(doc_list):
    texts = {}
    for doc in doc_list:
        texts[doc] = file_preprocessing(doc)
    return texts

In [17]:
def compute_sim(lemma, td_matrix, relevance_dict, avgdl, N):
    """
    Compute similarity score between search query and documents from collection
    :return: score
    """
    doc_list = td_matrix[td_matrix[lemma] > 0][lemma]
    n = len(doc_list)
    for doc in doc_list.index:
        n = doc_list[doc]
        dl = td_matrix.sum(axis=1)[doc]
        qf = n / dl
        relevance_dict[doc] += score_BM25(qf, dl, avgdl, k1, b, N, n)
    return relevance_dict


def get_search_result(query, td_matrix, doc_list):
    """
    Compute sim score between search query and all documents in collection
    Collect as pair (doc_id, score)
    :param query: input text
    :return: dict (doc_id, score)
    """
    relevance_dict = defaultdict(float)
    avgdl = get_avgdl(doc_list)
    query = text_preprocessing(query)
    for word in query:
        relevance_dict = compute_sim(word, td_matrix, relevance_dict, avgdl, len(doc_list))
    return relevance_dict

In [18]:
texts = get_texts(files_list)
td_matrix_df = pd.DataFrame(td_matrix).fillna(0)

In [19]:
res = get_search_result('рождественские каникулы', td_matrix_df, texts)
for key in sorted(res, key=lambda x: res[x], reverse=True):
    print(key, res[key])

Friends/Friends - season 2/Friends - 2x09 - The One With Phoebe's Dad.ru.txt 0.008803544377581326
Friends/Friends - season 6/Friends - 6x19 - The One With Joey's Fridge.ru.txt 0.007605017962981438
Friends/Friends - season 1/Friends - 1x17 - The One With Two Parts (2).ru.txt 0.006903455215778603
Friends/Friends - season 1/Friends - 1x16 - The One With Two Parts (1).ru.txt 0.006463741835326474
Friends/Friends - season 7/Friends - 7x10 - The One With The Holiday Armadillo.ru.txt 0.003879602825373096
Friends/Friends - season 4/Friends - 4x03 - The One With The 'Cuffs.ru.txt 0.003739257771240786
Friends/Friends - season 6/Friends - 6x09 - The One Where Ross Got High.ru.txt 0.0029575426967329995
